In [1]:
import json
import string
import re
from pprint import pprint
import pandas as pd
import numpy as np

In [2]:
numbers = "0123456789"
def toInt(instring):
    instring = str(instring)
    answer = ""
    for letter in instring:
        if letter in numbers:
            answer += letter
    return int(answer) if len(answer) > 0 and int(answer) < 100 else None
    return int(answer) if len(answer) > 0 else None

In [7]:
import re
re.search("(\d.)?\d+\s*M(\s||$)$" ,"22.1184 Mhz").group()

AttributeError: 'NoneType' object has no attribute 'group'

In [2]:
with open("Identifiers.json", "r") as read_file:
    indefecators = json.load(read_file)
orders = indefecators["Order"]
indefecators = indefecators["Identifiers"]


def is_this_column_(column, column_name):
    column_indefecator = indefecators[column_name]
    counter = 0
    for item in column:
        item = str(item)
        for indefecator in column_indefecator:
            search_pattern = f"\d+\s*{indefecator}" if column_name not in ["Tolerance", "Body"] else indefecator
            if re.search(search_pattern ,item) is not None:
                counter += 1
    return counter

FileNotFoundError: [Errno 2] No such file or directory: 'Identifiers.json'

In [3]:
def to_normal_order(cell, out_column_name):
    colum_order = orders[out_column_name]
    for order in colum_order:
        for order_pattern in colum_order[order]:
            search_pattern = f"\d+\s?{order_pattern}"
            founded = re.search(search_pattern ,cell)
            if founded is not None:
                return float(re.search("\d+", cell).group()) * float(order)
    return cell
    raise "Error"

In [469]:
def get_value(row, in_column_name, out_column_name):
    if in_column_name is None:
        return None
    if out_column_name == "Quantity":
        return None
    cell = str(row[in_column_name])
    column_indefecator = indefecators[out_column_name]
    for indefecator in column_indefecator:
        search_pattern = f"(\d.)?\d+\s*{indefecator}" if out_column_name not in ["Tolerance", "Body"] else indefecator
        founded = re.search(search_pattern ,cell)
        if founded is not None:
            if out_column_name in ["Capacity", "Resistance", "Inductance"]:
                return to_normal_order(founded.group(), out_column_name)
            return founded.group()
    return None

In [492]:
def get_value(row, in_column_name, out_column_name):
    if in_column_name is None:
        return None
    if out_column_name == "Quantity":
        return None
    cell = str(row[in_column_name])
    column_indefecator = indefecators[out_column_name]
    for indefecator in column_indefecator:
        search_pattern = f"(\d.)?\d+\s*{indefecator}" if out_column_name not in ["Tolerance", "Body"] else indefecator
        founded = re.search(search_pattern ,cell)
        if founded is not None:
            return founded.group()
    return None

In [308]:
def is_order_number(row):
    i = int(len(row)/2)
    if str(row[i]).isnumeric() and str(row[i+1]).isnumeric() and str(row[i-1]).isnumeric():
        if (int(row[i]) - int(row[i-1]))*(int(row[i+1]) - int(row[i])) == 1:
            return True
    return False

def is_quantity(row):
    counter = 0
    for i in row:
        i = str(i)
        if i.isnumeric() and int(float(i)) < 101:
            counter += 1
    return (not is_order_number(row)) * counter

In [297]:
def check_type(data):
    if ("Capacity" in data) and ("Tolerance" in data) and ("Body" in data):
        return "Capacitor"
    if ("Resistance" in data) and ("Body" in data):
        return "Resistor"
    return "Unkown type"

In [1]:
with open("Chips.json", "r") as read_file:
    chips = json.load(read_file)["Other"]

def maybe_this_is_special_board(name):
    for ch in chips:
        if ch in name:
            return ch, "Other"
    return None

FileNotFoundError: [Errno 2] No such file or directory: 'Chips.json'

In [488]:
class Specification():
    def __init__(self, path, rows_to_skip = 0, columns_to_work_with = None):
        
        ###
        if columns_to_work_with is not None:
            self.data = pd.read_excel(path, dtype=str)[columns_to_work_with][rows_to_skip:]
        else:
            self.data = pd.read_excel(path, dtype=str)[rows_to_skip:]
        ###
        
        self.path = path
        
        self.parsed_data = pd.DataFrame(
            columns=["Component_type",
                     "Name",
                     "Capacity",
                     "Resistance",
                     "Inductance",
                     "Tolerance",
                     "Body",
                     "Quantity",
                    ])
        
        self.conformity = {
            "Component_type": None,
            "Name": None,
            "Capacity": None,
            "Resistance": None,
            "Inductance": None,
            "Tolerance": None,
            "Body": None,
            "Quantity": None,
        }
        
        self.make_conformity()
        
    def make_conformity(self):
        columns = {}
        for col in self.data.columns:
            for key, value in indefecators.items():
                reason = is_this_column_(self.data[col],key)
                if reason > 0:
                    if key not in columns:
                        columns[key] = []
                    columns[key].append( (col, reason))
                    
        for col in self.data.columns:
            reason = is_quantity(self.data[col])
            if reason > 0:
                if "Quantity" not in columns:
                    columns["Quantity"] = []
                columns["Quantity"].append( (col, reason))
            
        pprint(columns)
        for col in columns:
            columns[col] = sorted(columns[col],key= lambda x: x[1],reverse=True)
            self.conformity[col] = [columns[col][0][0]]
    
    def change_conformity(self, input_col: str, output_col: str):
        self.conformity[output_col] = [input_col]
                
    def return_parsed(self):
        if len(self.parsed_data) == 0:
            self.parse()
        return self.parsed_data()
    
    def parse(self):
        for i in range(len(self.data)):
            # print(f"Now parse {i}")
            self.parse_item(i)
            
    def parse_item(self, i):
        current_row = self.data.iloc[i]
        parsed_row = {}
        for col in ["Capacity","Resistance","Inductance","Tolerance","Body","Quantity"]:
            if self.conformity[col] is not None:
                value = get_value(current_row, self.conformity[col][0], col)
                if value is not None:
                    parsed_row[col] = value
                    
        # if everything is none get all in name
        if len(parsed_row) == 0:
            name = ""
            for part in current_row:
                if part is not np.nan:
                    name += str(part) + " "
            parsed_row["Name"] = name[:-1]
        # check type
        parsed_row["Component_type"] = check_type(parsed_row)
        if self.conformity["Quantity"] is not None:
            # Здесь превращать в int
            parsed_row["Quantity"] = toInt(current_row[self.conformity["Quantity"][0]])
        else:
            parsed_row["Quantity"] = 1
        self.parsed_data = self.parsed_data.append(parsed_row, ignore_index=True)
        
    def save_parsed(self, name = None):
        if len(self.parsed_data) == 0:
            self.parse() 
        if name is None:
            name = "parsed_Specification.xlsx"
        self.parsed_data.to_excel(name,merge_cells=False)
        
    def save_parsed_grouped(self, name = None):
        if len(self.parsed_data) == 0:
            self.parse() 
        if name is None:
            name = "parsed_groped_Specification.xlsx"
        self.parsed_data.Quantity = self.parsed_data.Quantity.fillna(0)
        self.parsed_data.fillna('-').groupby([
                                                "Component_type",
                                                "Name",
                                                "Capacity",
                                                "Resistance",
                                                "Inductance",
                                                "Tolerance",
                                                "Body"
                                            ])["Quantity"].sum().to_excel(name, merge_cells=False)

In [495]:
test_spec = Specification("data/Плата CRIER3. Парсинг.xlsx")
#test_spec.change_conformity("Qty","Quantity") # Столбец с количеством мы спрашиваем
test_spec.conformity

{'Body': [('Unnamed: 4', 16)],
 'Capacity': [('Плат всего', 8), (60, 1)],
 'Inductance': [('Плат всего', 2)],
 'Quantity': [('Плат всего', 2), ('Unnamed: 5', 30)],
 'Resistance': [('Плат всего', 5), (60, 2)],
 'Tolerance': [('Плат всего', 8), (60, 14)]}


{'Component_type': None,
 'Name': None,
 'Capacity': ['Плат всего'],
 'Resistance': ['Плат всего'],
 'Inductance': ['Плат всего'],
 'Tolerance': [60],
 'Body': ['Unnamed: 4'],
 'Quantity': ['Unnamed: 5']}

In [496]:
test_spec.save_parsed()

In [486]:
test_spec.save_parsed_grouped()

In [484]:
test_spec.data

Unnamed: 0                                       Плата CRIER3  \
0           №                                         Designator   
1           1                                                 U3   
2           2                                                 U2   
3           3                                                VD1   
4           4                                                 D1   
5           5                                             C8, C9   
6           6             C12, C13, C18, C19, C20, C21, C22, C23   
7           7                                             C2, C6   
8           8                                                C48   
9           9  C3, C5, C7, C11, C15, C16, C17, C24, C26, C27,...   
10         10  C1, C4, C10, C14, C28, C33, C36, C38, C41, C42...   
11         11                                           C25, C46   
12         12                                                R41   
13         13  R13, R19, R21, R22, R23, R30, R31, R32, R33, R...   
14         14                                           R17, R42   
15         15  R1, R2, R25, R26, R27, R28, R29, R34, R38, R39...   
16         16                                                R16   
17         17                                R3, R4, R5, R6, R14   
18         18                                                RPI   
19         19                                           VD2, VD4   
20         20                                           C39, C40   
21         21                                                 Q1   
22         22                                                 Q2   
23         23                                                 L1   
24         24                                                 L2   
25         25                                                 L3   
26         26                                                VD3   
27         27                                                ZQ1   
28         28                                                 D2   
29         29                                                 U1   
30         30                                                 D3   

          Плат всего              60           Unnamed: 4 Unnamed: 5  
0             LibRef           Прим.               Корпус   Quantity  
1          MCP16301T             NaN             SOT23-G6          1  
2        CS42448-CQZ             NaN              TQFP-64          1  
3            SMAJ24A             NaN                  SMA          1  
4       STM32L431CCT             NaN  TSQFP50P900X900-48L          1  
5               18pF         NPO 50V                 0603          2  
6              2.7nF         NPO 50V                 0603          8  
7              2.2uF         25V X7R                 0603          2  
8                1nF         50V X7R                 0603          1  
9              0.1uF         50V X7R                 0603         20  
10              10uF         25V X7R                 0805         12  
11             4.7uF         X7R 50V                 1210          2  
12             31.6k              1%                 0603          1  
13                24              5%                 0603         12  
14               10k              1%                 0603          2  
15              4.7k              5%                 0603         12  
16                1k              1%                 0603          1  
17                 0              1%                 0603          5  
18             PBD40             NaN                PBD40          1  
19           MBR0540             NaN               SOD123          2  
20             100uF            6.3V    CAPT3528 (B CASE)          2  
21            BSS138             NaN             SOT23-G3          1  
22         IRLML6402             NaN             SOT23-G3          1  
23    BLM18PG181SN1D             NaN                 0603          1  
24             4.7uH  LQH43MN4R7K03L               

In [280]:
for cell in set(test_spec.data.Tolerance):
    cell = str(cell)
    column_indefecator = indefecators["Tolerance"]
    for indefecator in column_indefecator:
        search_pattern =  indefecator
        founded = re.search(search_pattern ,cell)
        if founded is not None:
            print(f"{cell} -> {founded.group()}")

0.2 -> 0.2
0.01 -> 0.01
0.05 -> 0.05
0.1 -> 0.1


In [281]:
set(test_spec.parsed_data.Tolerance)

{'0.01', '0.05', '0.1', '0.2', nan}

In [282]:
pd.DataFrame(test_spec.parsed_data.fillna('-').groupby([
    "Name",
    "Component_type",
    "Capacity",
    "Resistance",
    "Inductance",
    "Tolerance",
    "Body"
])["Quantity"].sum())

Quantity
Name                                               Component_type Capacity Resistance Inductance Tolerance Body         
-                                                  Capacitor      0.01uf   -          -          0.2       1206        1
                                                                  0.1uf    -          -          0.1       0603       12
                                                                                                 0.2       0603        2
                                                                                                           0805       12
                                                                                                           1206        2
...                                                                                                                  ...
97 1 R89 MF-R065 PTC Resettable Fuses Bourns Inc.  Unkown type    -        -          -          -         -           1
98 1 S21 IT-1102W8-160G-G Tactile Switch Switronic Unkown type    -        -          -          -         -           1
99 22 S1-20 S41-42 Tactile Switch                  Unkown type    -        -          -          -         -          22
Qty Левое крыло Value Tolerance Part Number Des... Unkown type    -        -          -          -         -           -
Qty Правое крыло Value Tolerance Part Number De... Unkown type    -        -          -          -         -           -

[121 rows x 1 columns]

In [283]:
test_spec.save_parsed_grouped()

In [14]:
    "Name",
    "Capacity",
    "Resistance",
    "Inductance",
    "Tolerance",
    "Body"

'Body'

In [413]:
files = [
    "data/Спецификации SNRM-7140.xlsx",
    "data/Спецификация_CRIER3_3_Дозакупка_21_05_20_05_20.xls",
    "data/SolderBOM-SNINTCOM2.xls",
    "data/SNA_8521_C_измененная_Автосохраненный.xlsx",
    "data/Спецификация_SFS_3381_v3_Автосохраненный.xlsx"
]

In [414]:
parsed = []
for i, path in enumerate(files):
    #test_spec = Specification(path)
    #test_spec.parse()
    parsed.append(Specification(path))
    parsed[-1].parse()
    parsed[-1].save_parsed(f"out/{i}-parsed.xlsx")
    parsed[-1].parsed_data.Quantity = parsed[-1].parsed_data.Quantity.fillna(0)
    parsed[-1].save_parsed_grouped(f"out/{i}-parsed-groped.xlsx")

In [322]:
parsed[0].parsed_data.fillna('-').groupby([
    "Name",
    "Component_type",
    "Capacity",
    "Resistance",
    "Inductance",
    "Tolerance",
    "Body"
])["Quantity"].sum().to_excel("test.xlsx")

In [251]:
parsed[1].parsed_data

Component_type                                               Name Capacity  \
0     Unkown type  1 4.5V to 30V 0.6A buck converter MCP16301T SO...      NaN   
1     Unkown type  2 108 dB, 192 kHz 6-In, 8-Out Audio codec CS42...      NaN   
2     Unkown type  3 400W Surface mount transient voltage suppres...      NaN   
3     Unkown type                                                NaN      NaN   
4       Capacitor                                                NaN     18pF   
5       Capacitor                                                NaN      7nF   
6       Capacitor                                                NaN      2uF   
7       Capacitor                                                NaN      1nF   
8       Capacitor                                                NaN    0.1uF   
9       Capacitor                                                NaN     10uF   
10       Resistor                                                NaN      7uF   
11       Resistor                                                NaN      NaN   
12    Unkown type                                                NaN      NaN   
13       Resistor                                                NaN      NaN   
14       Resistor                                                NaN      NaN   
15       Resistor                                                NaN      NaN   
16    Unkown type                                                NaN      NaN   
17    Unkown type  18 Connector, 20-Pin, Dual row PBD40 PBD40 RPI...      NaN   
18    Unkown type  19 Diode Surface Mount MBR0540 SOD123 VD2, VD4...      NaN   
19    Unkown type                                                NaN    100uF   
20    Unkown type  21 Generic N-channel MOSFET BSS138 SOT23-G3 Q1...      NaN   
21    Unkown type  22 Generic P-channel MOSFET IRLML6402 SOT23-G3...      NaN   
22    Unkown type                                                NaN      NaN   
23       Resistor                                                NaN      NaN   
24    Unkown type                                                NaN      NaN   
25    Unkown type  26 Schottky diode BAT54WS SOD323 VD3 1 100 BAT...      NaN   
26    Unkown type                                                NaN      NaN   
27    Unkown type  28 Standard pinout SO8 serial flash MT25QL128A...      NaN   
28    Unkown type  29 Stereo 24-bits audio ADC with 4x2 mux PCM18...      NaN   
29    Unkown type  30 TI's SPDT low power switch TS5A3157DBV SOT2...      NaN   

   Resistance Inductance Tolerance  Body Quantity  
0         NaN        NaN       NaN   NaN        1  
1         NaN        NaN       NaN   NaN        1  
2         NaN        NaN       NaN   NaN        1  
3         NaN        NaN     0P900   NaN        1  
4         NaN        NaN      0603  0603        2  
5         NaN        NaN      0603  0603        8  
6        225K        NaN      0603  0603        2  
7       0603K        NaN      0603  0603        1  
8      0B104K        NaN      0603  0603       20  
9        106K        NaN      0805  0805       12  
10       475K        NaN       NaN  1210        2  
11      0731K        NaN      0603  0603        1  
12        NaN        NaN      0603  0603       12  
13        10K        NaN      0603  0603        2  
14         4K        NaN      0603  0603       12  
15         1K        NaN      0603  0603        1  
16        NaN        NaN      0603  0603        5  
17        NaN        NaN       NaN   NaN        1  
18        NaN        NaN       NaN   NaN        2  
19       107K        NaN       NaN   NaN        2  
20        NaN        NaN       NaN   NaN        1  
21        NaN        NaN       NaN   NaN        1  
22        NaN        NaN      0603  0603        1  
23         7K      7e-06       NaN  1812        1  
24        NaN    1.5e-05       NaN   NaN        1  
25        NaN        NaN       NaN   NaN        1  
26      1184M        NaN     03225   NaN        1  
27        NaN        NaN       NaN 

In [312]:
# Это итог
final = pd.concat([spec.parsed_data for spec in parsed])
set(final.Quantity)

{0,
 1,
 10,
 11,
 12,
 14,
 15,
 16,
 17,
 18,
 2,
 20,
 22,
 23,
 25,
 26,
 27,
 28,
 29,
 3,
 35,
 39,
 4,
 46,
 47,
 5,
 51,
 6,
 7,
 75,
 8,
 9,
 None}

In [303]:
set(final.Quantity)

{0,
 1,
 10,
 100,
 1052020,
 11,
 1102,
 12,
 121,
 1282,
 138,
 14,
 15,
 16,
 17,
 18,
 181811,
 18884,
 2,
 20,
 202,
 20200501000000,
 2151,
 22,
 220,
 23,
 25,
 251281,
 26,
 261,
 27,
 28,
 287,
 29,
 298518,
 298530,
 3,
 3203101,
 32745,
 331,
 337,
 35,
 370001,
 386,
 39,
 4,
 46,
 47,
 499,
 5,
 502,
 503,
 51,
 511,
 540,
 5501,
 6,
 7,
 7342,
 7404,
 75,
 7986,
 8,
 828040264210,
 8413,
 8415,
 847,
 857,
 8720,
 888,
 9,
 None}

In [313]:
final.Quantity = final.Quantity.fillna(0)

In [330]:
final.fillna('-').groupby([
                    "Component_type",
                    "Name",
                    "Capacity",
                    "Resistance",
                    "Inductance",
                    "Tolerance",
                    "Body"
                ])["Quantity"].sum().to_excel('final.xlsx')

In [38]:
df1 = pd.DataFrame([('cat',2,"a"),("dog",4,"a"),("dog",6,"b")], columns=['a','b','c'])
df2 = pd.DataFrame([("cat",200,"a"),("dog",400,"a"),("dog",600,"b")], columns=['a','b','c'])
df1

a  b  c
0  cat  2  a
1  dog  4  a
2  dog  6  b

In [39]:
df2

a    b  c
0  cat  200  a
1  dog  400  a
2  dog  600  b

In [40]:
df_add = df1.add(df2, fill_value=0,axis='b')
df_add

a    b   c
0  catcat  202  aa
1  dogdog  404  aa
2  dogdog  606  bb

In [41]:
pd.merge(df1,df2,left_on=['a','c'],right_on=['a','c'],how='left')

a  b_x  c  b_y
0  cat    2  a  200
1  dog    4  a  400
2  dog    6  b  600

In [36]:
df1.loc[:, 'a'] = df2 

In [46]:
float("1")

1.0

In [47]:
test_spec.data

Индикатор Unnamed: 1            Unnamed: 2 Unnamed: 3  \
0          №        NaN                   NaN     Кол-во   
1          1        DA1              L7812ABV          1   
2          2     R1-R15  CF-25 (С1-4) 0.25 Вт         15   
3        NaN        R16   KNP-200 2 Вт, 10 Ом          1   
4          3      C1-C4           47 мкФх35 В          4   
..       ...        ...                   ...        ...   
75        20        NaN                   NaN        NaN   
76        21        NaN                   NaN        NaN   
77        22        NaN                   NaN        NaN   
78        23         X1                   NaN          1   
79        24        NaN                   NaN        NaN   

                                    Unnamed: 4  \
0                                          NaN   
1   Стабилизатор напряжения +12В 1.5А [TO-220]   
2                        Резистор углеродистый   
3                         Резистор проволочный   
4                Конденсатор электролитический   
..                                         ...   
75                                         NaN   
76                                         NaN   
77                                         NaN   
78             SCM-20 (DS1011-20S) (IDCC-20MS)   
79                                         NaN   

                                 Unnamed: 5 Unnamed: 6  
0                                       NaN        NaN  
1   https://www.chipdip.ru/product/l7812abv          +  
2     https://www.chipdip.ru/product0/44435        NaN  
3     https://www.chipdip.ru/product0/27855        NaN  
4                                       NaN          +  
..                                      ...        ...  
75                                      NaN        NaN  
76                                      NaN        NaN  
77                                      NaN        NaN  
78                  Вилка прямая с защелкой        NaN  
79                                      NaN        NaN  

[80 rows x 7 columns]

In [90]:
s1 = test_spec.data["Индикатор"]
s2 = test_spec.data["Unnamed: 3"]

In [98]:
def give_stats(row):
    num = []
    for i in row:
        i = str(i)
        if i.isnumeric():
            num.append(int(i))
    return pd.DataFrame(num).describe()

In [176]:
def is_order_number(row):
    i = int(len(row)/2)
    if row[i].isnumeric() and row[i+1].isnumeric() and row[i-1].isnumeric():
        if (int(row[i]) - int(row[i-1]))*(int(row[i+1]) - int(row[i])) == 1:
            return True
    return False

def is_quantity(row):
    counter = 0
    for i in row:
        i = str(i)
        if i.isnumeric() and int(float(i)):
            counter += 1
    return not is_order_number(row) * counter

In [177]:
def is_quantity(row):
    counter = 0
    for i in row:
        i = str(i)
        if i.isnumeric() and int(float(i)):
            counter += 1
    return not is_order_number(row) * counter

In [209]:
orders

{'Capacity': {'0,1': [], '1': [], '10': []},
 'Resistance': {'0,1': [], '1': [], '10': []},
 'Inductance': {'1e-6': ['uH', 'uh', 'мкГн', 'мкгн'],
  '1': ['H', 'h', 'Гн', 'гн']}}

In [167]:
is_order_number(s2)

In [100]:
give_stats(s2)

0
count  60.000000
mean    3.050000
std     3.180582
min     1.000000
25%     1.000000
50%     1.500000
75%     4.000000
max    15.000000

In [94]:
np.max(num)

24

In [96]:
df_describe = pd.DataFrame(num).describe()
a =df_describe.describe()

In [97]:
a

0
count  60.000000
mean    8.966667
std     6.002730
min     1.000000
25%     4.000000
50%     8.000000
75%    12.000000
max    24.000000

In [107]:
orders

{'Capacity': {'0,1': [], '1': [], '10': []},
 'Resistance': {'0,1': [], '1': [], '10': []},
 'Inductance': {'1e-6': ['uH', 'uh', 'мкГн', 'мкгн'],
  '1': ['H', 'h', 'Гн', 'гн']}}

In [233]:
to_normal_order("1 H","Inductance")

<re.Match object; span=(0, 3), match='1 H'>
1.0 1.0


1.0

In [113]:
float("0.1")

0.1

In [419]:
class stock_manufacture():
    def __init__(self, path=None, data=None):
        if data is None:
            self.data = pd.read_excel(path).fillna("-")
        else:
            self.data = data
    
    def union(self, other):
        return stock_manufacture(data=pd.DataFrame(pd.concat([self.data, other.data]).fillna('-').groupby([
                    "Component_type",
                    "Name",
                    "Capacity",
                    "Resistance",
                    "Inductance",
                    "Tolerance",
                    "Body"
                ])["Quantity"].sum()))
        
    def minus(self, specification):
        # specification as pandas DF 
        estimate = pd.merge(specification,self.data,on=[
            "Component_type",
            "Name",
            "Capacity",
            "Resistance",
            "Inductance",
            "Tolerance",
            "Body"
        ],how='left',suffixes=("_specification", "_in_stock"))
        estimate["Quantity"] = estimate.Quantity_in_stock - estimate.Quantity_specification
        estimate = estimate.drop(columns=["Quantity_specification", "Quantity_in_stock"])
        return estimate.loc[estimate["Quantity"] > 0]

In [420]:
test = stock_manufacture(path="out/1-parsed-groped.xlsx")
test2 = stock_manufacture(path="out/1-parsed-groped.xlsx")
big_test = test.union(test2)

In [405]:
test.data

Component_type                                               Name Capacity  \
0       Capacitor                                                  -    0.1uF   
1             NaN                                                NaN     10uF   
2             NaN                                                NaN     18pF   
3             NaN                                                NaN      1nF   
4             NaN                                                NaN      2uF   
5             NaN                                                NaN      7nF   
6        Resistor                                                  -        -   
7             NaN                                                NaN      NaN   
8             NaN                                                NaN      NaN   
9             NaN                                                NaN      NaN   
10            NaN                                                NaN      NaN   
11            NaN                                                NaN      7uF   
12    Unkown type                                                  -        -   
13            NaN                                                NaN      NaN   
14            NaN                                                NaN      NaN   
15            NaN                                                NaN      NaN   
16            NaN                                                NaN    100uF   
17            NaN  1 4.5V to 30V 0.6A buck converter MCP16301T SO...        -   
18            NaN  18 Connector, 20-Pin, Dual row PBD40 PBD40 RPI...        -   
19            NaN  19 Diode Surface Mount MBR0540 SOD123 VD2, VD4...        -   
20            NaN  2 108 dB, 192 kHz 6-In, 8-Out Audio codec CS42...        -   
21            NaN  21 Generic N-channel MOSFET BSS138 SOT23-G3 Q1...        -   
22            NaN  22 Generic P-channel MOSFET IRLML6402 SOT23-G3...        -   
23            NaN  26 Schottky diode BAT54WS SOD323 VD3 1 100 BAT...        -   
24            NaN  28 Standard pinout SO8 serial flash MT25QL128A...        -   
25            NaN  29 Stereo 24-bits audio ADC with 4x2 mux PCM18...        -   
26            NaN  3 400W Surface mount transient voltage suppres...        -   
27            NaN  30 TI's SPDT low power switch TS5A3157DBV SOT2...        -   

   Resistance Inductance Tolerance  Body  Quantity  
0      0B104K          -      0603  0603        20  
1        106K          -      0805  0805        12  
2           -          -      0603  0603         2  
3       0603K          -      0603  0603         1  
4        225K          -      0603  0603         2  
5           -          -      0603  0603         8  
6       0731K          -      0603  0603         1  
7         10K          -      0603  0603         2  
8          1K          -      0603  0603         1  
9          4K          -      0603  0603        12  
10         7K      7e-06         -  1812         1  
11       475K          -         -  1210         2  
12          -    1.5e-05         -     -         1  
13        NaN          -      0603  0603        18  
14        NaN        NaN     0P900     -         1  
15      1184M          -     03225     -         1  
16       107K          -         -     -         2  
17          -          -         -     -         1  
18          -          -         -     -         1  
19          -          -         -     -         2  
20          -          -         -     -         1  
21          -          -         -     -         1  
22          -          -         -     -         1  
23          -          -         -     -         1  
24          -          -         -     -         1  
25          -          -         -     -         1  
26          -          -         -     -         1  
27          -          -         -     -         1

In [421]:
test.data

Component_type                                               Name Capacity  \
0       Capacitor                                                  -    0.1uF   
1       Capacitor                                                  -     10uF   
2       Capacitor                                                  -     18pF   
3       Capacitor                                                  -      1nF   
4       Capacitor                                                  -      2uF   
5       Capacitor                                                  -      7nF   
6        Resistor                                                  -        -   
7        Resistor                                                  -        -   
8        Resistor                                                  -        -   
9        Resistor                                                  -        -   
10       Resistor                                                  -        -   
11       Resistor                                                  -      7uF   
12    Unkown type                                                  -        -   
13    Unkown type                                                  -        -   
14    Unkown type                                                  -        -   
15    Unkown type                                                  -        -   
16    Unkown type                                                  -    100uF   
17    Unkown type  1 4.5V to 30V 0.6A buck converter MCP16301T SO...        -   
18    Unkown type  18 Connector, 20-Pin, Dual row PBD40 PBD40 RPI...        -   
19    Unkown type  19 Diode Surface Mount MBR0540 SOD123 VD2, VD4...        -   
20    Unkown type  2 108 dB, 192 kHz 6-In, 8-Out Audio codec CS42...        -   
21    Unkown type  21 Generic N-channel MOSFET BSS138 SOT23-G3 Q1...        -   
22    Unkown type  22 Generic P-channel MOSFET IRLML6402 SOT23-G3...        -   
23    Unkown type  26 Schottky diode BAT54WS SOD323 VD3 1 100 BAT...        -   
24    Unkown type  28 Standard pinout SO8 serial flash MT25QL128A...        -   
25    Unkown type  29 Stereo 24-bits audio ADC with 4x2 mux PCM18...        -   
26    Unkown type  3 400W Surface mount transient voltage suppres...        -   
27    Unkown type  30 TI's SPDT low power switch TS5A3157DBV SOT2...        -   

   Resistance Inductance Tolerance  Body  Quantity  
0      0B104K          -      0603  0603        20  
1        106K          -      0805  0805        12  
2           -          -      0603  0603         2  
3       0603K          -      0603  0603         1  
4        225K          -      0603  0603         2  
5           -          -      0603  0603         8  
6       0731K          -      0603  0603         1  
7         10K          -      0603  0603         2  
8          1K          -      0603  0603         1  
9          4K          -      0603  0603        12  
10         7K      7e-06         -  1812         1  
11       475K          -         -  1210         2  
12          -    1.5e-05         -     -         1  
13          -          -      0603  0603        18  
14          -          -     0P900     -         1  
15      1184M          -     03225     -         1  
16       107K          -         -     -         2  
17          -          -         -     -         1  
18          -          -         -     -         1  
19          -          -         -     -         2  
20          -          -         -     -         1  
21          -          -         -     -         1  
22          -          -         -     -         1  
23          -          -         -     -         1  
24          -          -         -     -         1  
25          -          -         -     -         1  
26          -          -         -     -         1  
27          -          -         -     -         1

In [422]:
big_test.minus(test.data)

Component_type                                               Name Capacity  \
0       Capacitor                                                  -    0.1uF   
1       Capacitor                                                  -     10uF   
2       Capacitor                                                  -     18pF   
3       Capacitor                                                  -      1nF   
4       Capacitor                                                  -      2uF   
5       Capacitor                                                  -      7nF   
6        Resistor                                                  -        -   
7        Resistor                                                  -        -   
8        Resistor                                                  -        -   
9        Resistor                                                  -        -   
10       Resistor                                                  -        -   
11       Resistor                                                  -      7uF   
12    Unkown type                                                  -        -   
13    Unkown type                                                  -        -   
14    Unkown type                                                  -        -   
15    Unkown type                                                  -        -   
16    Unkown type                                                  -    100uF   
17    Unkown type  1 4.5V to 30V 0.6A buck converter MCP16301T SO...        -   
18    Unkown type  18 Connector, 20-Pin, Dual row PBD40 PBD40 RPI...        -   
19    Unkown type  19 Diode Surface Mount MBR0540 SOD123 VD2, VD4...        -   
20    Unkown type  2 108 dB, 192 kHz 6-In, 8-Out Audio codec CS42...        -   
21    Unkown type  21 Generic N-channel MOSFET BSS138 SOT23-G3 Q1...        -   
22    Unkown type  22 Generic P-channel MOSFET IRLML6402 SOT23-G3...        -   
23    Unkown type  26 Schottky diode BAT54WS SOD323 VD3 1 100 BAT...        -   
24    Unkown type  28 Standard pinout SO8 serial flash MT25QL128A...        -   
25    Unkown type  29 Stereo 24-bits audio ADC with 4x2 mux PCM18...        -   
26    Unkown type  3 400W Surface mount transient voltage suppres...        -   
27    Unkown type  30 TI's SPDT low power switch TS5A3157DBV SOT2...        -   

   Resistance Inductance Tolerance  Body  Quantity  
0      0B104K          -      0603  0603        20  
1        106K          -      0805  0805        12  
2           -          -      0603  0603         2  
3       0603K          -      0603  0603         1  
4        225K          -      0603  0603         2  
5           -          -      0603  0603         8  
6       0731K          -      0603  0603         1  
7         10K          -      0603  0603         2  
8          1K          -      0603  0603         1  
9          4K          -      0603  0603        12  
10         7K      7e-06         -  1812         1  
11       475K          -         -  1210         2  
12          -    1.5e-05         -     -         1  
13          -          -      0603  0603        18  
14          -          -     0P900     -         1  
15      1184M          -     03225     -         1  
16       107K          -         -     -         2  
17          -          -         -     -         1  
18          -          -         -     -         1  
19          -          -         -     -         2  
20          -          -         -     -         1  
21          -          -         -     -         1  
22          -          -         -     -         1  
23          -          -         -     -         1  
24          -          -         -     -         1  
25          -          -         -     -         1  
26          -          -         -     -         1  
27          -          -         -     -         1

In [423]:
test.data

Component_type                                               Name Capacity  \
0       Capacitor                                                  -    0.1uF   
1       Capacitor                                                  -     10uF   
2       Capacitor                                                  -     18pF   
3       Capacitor                                                  -      1nF   
4       Capacitor                                                  -      2uF   
5       Capacitor                                                  -      7nF   
6        Resistor                                                  -        -   
7        Resistor                                                  -        -   
8        Resistor                                                  -        -   
9        Resistor                                                  -        -   
10       Resistor                                                  -        -   
11       Resistor                                                  -      7uF   
12    Unkown type                                                  -        -   
13    Unkown type                                                  -        -   
14    Unkown type                                                  -        -   
15    Unkown type                                                  -        -   
16    Unkown type                                                  -    100uF   
17    Unkown type  1 4.5V to 30V 0.6A buck converter MCP16301T SO...        -   
18    Unkown type  18 Connector, 20-Pin, Dual row PBD40 PBD40 RPI...        -   
19    Unkown type  19 Diode Surface Mount MBR0540 SOD123 VD2, VD4...        -   
20    Unkown type  2 108 dB, 192 kHz 6-In, 8-Out Audio codec CS42...        -   
21    Unkown type  21 Generic N-channel MOSFET BSS138 SOT23-G3 Q1...        -   
22    Unkown type  22 Generic P-channel MOSFET IRLML6402 SOT23-G3...        -   
23    Unkown type  26 Schottky diode BAT54WS SOD323 VD3 1 100 BAT...        -   
24    Unkown type  28 Standard pinout SO8 serial flash MT25QL128A...        -   
25    Unkown type  29 Stereo 24-bits audio ADC with 4x2 mux PCM18...        -   
26    Unkown type  3 400W Surface mount transient voltage suppres...        -   
27    Unkown type  30 TI's SPDT low power switch TS5A3157DBV SOT2...        -   

   Resistance Inductance Tolerance  Body  Quantity  
0      0B104K          -      0603  0603        20  
1        106K          -      0805  0805        12  
2           -          -      0603  0603         2  
3       0603K          -      0603  0603         1  
4        225K          -      0603  0603         2  
5           -          -      0603  0603         8  
6       0731K          -      0603  0603         1  
7         10K          -      0603  0603         2  
8          1K          -      0603  0603         1  
9          4K          -      0603  0603        12  
10         7K      7e-06         -  1812         1  
11       475K          -         -  1210         2  
12          -    1.5e-05         -     -         1  
13          -          -      0603  0603        18  
14          -          -     0P900     -         1  
15      1184M          -     03225     -         1  
16       107K          -         -     -         2  
17          -          -         -     -         1  
18          -          -         -     -         1  
19          -          -         -     -         2  
20          -          -         -     -         1  
21          -          -         -     -         1  
22          -          -         -     -         1  
23          -          -         -     -         1  
24          -          -         -     -         1  
25          -          -         -     -         1  
26          -          -         -     -         1  
27          -          -         -     -         1

In [497]:
def func(spec_data, stores):
    
    pass

TypeError: unsupported operand type(s) for -: 'list' and 'list'